In [73]:
import xml.etree.ElementTree as ET
import pandas as pd

In [74]:
data = pd.read_csv('../podatoci/mkcitiesborderbox.csv')

municipality = "debar"

In [105]:
def convert_osm_to_csv(all_path='', borderbox_path='', place=''):
    
    res = pd.DataFrame(columns=['WayIndex', 'WayId', 'NodeIndex', 'NodeId', 'Longitude', 'Latitude', 'Place'])
    
    all_root = ET.parse(all_path).getroot()
    borderbox_root = ET.parse(borderbox_path).getroot()

    relations = borderbox_root.findall('relation')
    if len(relations) > 1:
        raise Exception
        
    ways_ids = pd.DataFrame(columns=['WayIndex', 'WayId'])
    ind = 0
    for member in relations[0]:
        if member.get('type') == 'way':
            ind = ind + 1
            ways_ids = ways_ids.append({'WayIndex': ind, 'WayId': member.get('ref')}, ignore_index=True)
    
    ways = pd.DataFrame(columns=['WayId', 'NodeIndex', 'NodeId'])
    for way in root.findall('way'):
        if way.get('id') in set(ways_ids['WayId']):
            ind = 0
            for nd in way.findall('nd'):
                ind = ind + 1
                ways = ways.append({'WayId': way.get('id'), 'NodeIndex': ind, 'NodeId': nd.get('ref')}, ignore_index=True)
    
    ways = ways_ids.merge(ways, left_on='WayId', right_on='WayId')
    
    nodes = pd.DataFrame(columns=['NodeId', 'Longitude', 'Latitude', 'Place'])
    for node in all_root.findall('node'):
        if node.get('id') in set(ways['NodeId']):
            nodes = nodes.append({'NodeId': node.get('id'), 'Longitude': node.get('lon'), 'Latitude': node.get('lat'), 'Place': place}, ignore_index=True)

    res = ways.merge(nodes)
    return res
    

In [111]:
res = convert_osm_to_csv('./municipality-all/centarzhupa.osm', './municipality-borderbox/centarzhupa.osm', 'Центар Жупа')

In [112]:
res = res.sort_values(by=['WayIndex', 'NodeIndex'])

res

,WayIndex,WayId,NodeIndex,NodeId,Longitude,Latitude,Place
0,1,227935281,1,4336877861,20.6626157,41.4488352,Центар Жупа
2,1,227935281,2,4336877860,20.6655339,41.4479506,Центар Жупа
3,1,227935281,3,4336877859,20.6681088,41.4434470,Центар Жупа
4,1,227935281,4,4336877858,20.6684522,41.4410020,Центар Жупа
5,1,227935281,5,2039713321,20.6715421,41.4399725,Центар Жупа
...,...,...,...,...,...,...,...
344,30,436211178,8,4336877865,20.6629805,41.4524378,Центар Жупа
345,30,436211178,9,4336877864,20.6618647,41.4512798,Центар Жупа
346,30,436211178,10,4336877863,20.6625298,41.4510225,Центар Жупа
347,30,436211178,11,4336877862,20.6627659,41.4502827,Центар Жупа


In [114]:
res.to_csv('../podatoci/mkcitiesborder.csv', index=False, mode='a')